In [1]:
import numpy as np
from datasets import load_dataset
import re

from Attention.attention_block import attention_block as ab
from Attention.attention_head import attention_head as ah

import base_transformer as bt

%load_ext autoreload
%autoreload 2

In [2]:
import feed_forward as ff
test_layer = ff.neuron_layer(100, 100, 'relu', 8, .0001, .001)

In [ ]:

test_embedding = np.random.normal(0, 1, size=(9,512))
test_head = ah(512, 64)

# print(np.sum(test_head.q))
# test_head.calculate_q_k_v(test_embedding)

test_head.masked_attention_score(test_embedding)

# print(test_head.W_v.shape)

In [88]:
test_block = ab(num_heads=8, input_dimension=512, head_output_dimension=64, block_output_dimension=128)

# test_block.W_o.shape

test_block.multi_head_attention(test_embedding)

# test_block.heads['head_1'].W_k.shape

# test_block.attention_scores['head_0']

array([[-0.42385301,  0.48160827,  0.38554254, ...,  2.88721421,
         2.92049123, -3.06866551],
       [-0.31568235,  0.78359117,  0.44682596, ...,  3.13389481,
         1.04940131, -2.93723271],
       [-1.95363009, -0.12947937,  0.71579553, ...,  1.91326054,
         0.56326398, -1.92793466],
       ...,
       [-1.36727774, -0.66270116, -0.02321105, ..., -1.22368246,
         0.63504052,  1.01472738],
       [-1.51618975, -0.9181758 , -2.06444104, ...,  1.22475852,
         0.38885346,  0.13169262],
       [-0.19788695,  0.47782417, -0.60076089, ...,  0.67496138,
         1.19890487,  0.97390729]])

In [5]:
# constants
START_TOKEN = '<START>'
END_TOKEN = '<END>'
NUM_SAMPLES = 10000
imdbDataset = load_dataset("stanfordnlp/imdb")

embeddingsFilepath = '/Users/josep/Desktop/Self/Learning/NLP/RNN/data/glove.6B.300d.txt'

# helper functions
def read_corpus(dataset):
    files = dataset["train"]["text"][:NUM_SAMPLES]
    return [[START_TOKEN] + [re.sub(r'[^\w]', '', w.lower()) for w in f.split(" ")] + [END_TOKEN] for f in files]


def embedding_for_vocab(filepath, words, dimensions):
    vocab_size = len(words)
    embeddings = np.zeros((vocab_size, dimensions))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in words.keys():
                index = words[word]
                embeddings[index] = np.array(vector)[:dimensions]
    return embeddings

imdbCorpus = read_corpus(imdbDataset)

corpusWords = [y for x in imdbCorpus for y in x]
corpusWords = list(set(corpusWords))
word2ind={}
for i in range(len(corpusWords)+1):
    word2ind[corpusWords[i-1]] = i
word2ind['<PAD>'] = 0
embeddings = embedding_for_vocab(embeddingsFilepath, word2ind, 300)

In [ ]:
test_transformer = bt.transformer(
    embeddings=embeddings, word2ind=word2ind
    , input_layer_shape=100, input_layer_activation='relu'
    , hidden_layer_shapes=[100,100,100], hidden_layer_activations=['relu', 'relu', 'relu']
    , output_layer_activation='softmax'
)

In [14]:
test_transformer.input_layer.activation

'relu'